In [42]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash

In [ ]:
!sudo yum install git-lfs -y

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/microsoft/bloom-deepspeed-inference-int8/
    

In [ ]:
!cd bloom-deepspeed-inference-int8 && git lfs pull

In [ ]:
!cd bloom-deepspeed-inference-int8 && git pull

In [ ]:
!pwd


#### Start triton offering

In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        7.7G     0  7.7G   0% /dev
tmpfs           7.7G  4.0K  7.7G   1% /dev/shm
tmpfs           7.7G  608K  7.7G   1% /run
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  140G   89G   52G  64% /
/dev/nvme2n1    985G  357G  578G  39% /home/ec2-user/SageMaker
tmpfs           1.6G     0  1.6G   0% /run/user/1000
tmpfs           1.6G     0  1.6G   0% /run/user/1001
tmpfs           1.6G     0  1.6G   0% /run/user/1002


### Start Triton Server

**Note**: Amazon SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`. Below is the sample model directory structure

```
bloom-176b
├── 1
│   └── model.pt
│   └── model.py
└── config.pbtxt
```

tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude="*.bin" -exclude "*.tar" -zcvf model.tar.gz bert-gptj



In [202]:
!mkdir -p ./bloom-176b
!mkdir -p ./bloom-176b/1

In [203]:
%%writefile ./bloom-176b/1/model.pt
print("hello model pt")

Overwriting ./bloom-176b/1/model.pt


#### Following this configuration 

https://github.com/triton-inference-server/python_backend

Number of Inputs and # of Outputs have to be the same in triton 

for Testing or running Triton locally or to look at the configurations -- use -- https://chroniclesofai.com/mlops-chapter-8-model-server-with-nvidia-triton-local-part-1-b/

    
**Below is the RAW file with 2 inputs and outputs**    

**But we wil test with just 1 input and output**

In [221]:
%%writefile ./bloom-176b/config.pbtxt

name: "bloom-176b"
backend: "python"
max_batch_size: 32
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [512]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_INT32
    dims: [ 512 ]
  }
]


dynamic_batching {
   preferred_batch_size: 16
   max_queue_delay_microseconds: 1000
 }

Overwriting ./bloom-176b/config.pbtxt


In [222]:
import sys
import subprocess
reqs = subprocess.check_output([sys.executable, '-m', 'pip','freeze'])
#[r.decode().split('==')[0] for r in reqs.split()]

In [262]:
%%writefile ./bloom-176b/1/model.py

import numpy as np
import sys
import os
import json
from pathlib import Path

import sys
import subprocess
import traceback

# !pip install transformers
# !pip install sentencepiece
# !pip install nvidia-pyindex
# !pip install tritonclient[http]
# !pip install torch
# implement pip as a subprocess:
subprocess.check_call([sys.executable, '-m', 'pip', 'install','torch'])

# process output with an API in the subprocess module:
reqs = subprocess.check_output([sys.executable, '-m', 'pip','freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

print(installed_packages)

import torch

import triton_python_backend_utils as pb_utils

class TritonPythonModel:
    # Every Python model must have "TritonPythonModel" as the class name!
    def initialize(self, args):
        print(f"TritonPythonModel:Bloom-176b:initialize called:args={args}", flush=True)
        self.output_dtype = pb_utils.triton_string_to_numpy(
            pb_utils.get_output_config_by_name(
                json.loads(args['model_config']), "OUTPUT__0" #"output"
            )['data_type']
        )
        print(f"TritonPythonModel:Bloom-176b:initialize::output_type={self.output_dtype}", flush=True)
        
        #from transformers import T5ForConditionalGeneration, T5Tokenizer
        self.model = None #T5ForConditionalGeneration.from_pretrained("t5-small").cuda()
        
        try:
            from os import listdir
            from os.path import isfile, join
            onlyfiles = [f for f in listdir(".") if isfile(join('.', f))]
            print(f"TritonPythonModel:Bloom-176b:onlyfiles::{onlyfiles}", flush=True)
        except:
            pass
            
        print(f"TritonPythonModel:Bloom-176b:initialized", flush=True)

    def execute(self, requests):
        print(f"TritonPythonModel:Bloom-176b: execute called:requests={requests}:", flush=True)
        
        responses = []
        try:
            for request in requests:
                print(f"TritonPythonModel:Bloom-176b: execute :EACH:Request:requests={request}:", flush=True)
                
                input_0 = pb_utils.get_input_tensor_by_name(request, "INPUT__0")
                print(f"TritonPythonModel:Bloom-176b: execute :EACH:Request:input_0={input_0}::")
                try:
                    input_ids = input_0.as_numpy()
                    
                     # Convert to numpy array on cpu:
                    input_ids = torch.as_tensor(input_ids).long().cuda()
                    summary = self.model.generate(input_ids, num_beams=1)
                    np_summary = summary.cpu().int().detach().numpy()
                except:
                    pass
                
                summary = [1]*512  

               
                
                np_summary = np.array(summary)
                inference_response = pb_utils.InferenceResponse(
                    output_tensors=[
                        pb_utils.Tensor(
                            "OUTPUT__0",
                            np_summary.astype(self.output_dtype)
                        ),
                    ]
                )
                responses.append(inference_response)
                print(f"TritonPythonModel:Bloom-176b: execute():responses={responses}::{len(responses)}::")
        except :
            print("TritonPythonModel:Bloom-176b: execute():Exception Thrown", flush=True)
            print (f"TritonPythonModel:Bloom-176b: execute(): traceback:eror:predict={traceback.format_exc()}", flush=True)
            
        return responses

    def finalize(self):
        print(f"TritonPythonModel:Bloom-176b: finalize()::TritonPythonModel finalized: called", flush=True)


Overwriting ./bloom-176b/1/model.py


In [263]:
!cd bloom-176b && rm model.tar.gz
!rm model.tar.gz
!tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude=".bin" --exclude ".tar" -zcvf model.tar.gz bloom-176b  
!tar -tvf model.tar.gz


rm: cannot remove ‘model.tar.gz’: No such file or directory
bloom-176b/
bloom-176b/1/
bloom-176b/1/model.py
bloom-176b/1/.ipynb_checkpoints/
bloom-176b/1/.ipynb_checkpoints/model-checkpoint.py
bloom-176b/1/.ipynb_checkpoints/model-checkpoint.pt
bloom-176b/1/model.pt
bloom-176b/.ipynb_checkpoints/
bloom-176b/.ipynb_checkpoints/config-checkpoint.pbtxt
bloom-176b/config.pbtxt
drwxrwxr-x ec2-user/ec2-user 0 2022-10-11 04:22 bloom-176b/
drwxrwxr-x ec2-user/ec2-user 0 2022-10-10 20:09 bloom-176b/1/
-rw-rw-r-- ec2-user/ec2-user 3697 2022-10-13 00:11 bloom-176b/1/model.py
drwxrwxr-x ec2-user/ec2-user    0 2022-10-10 20:09 bloom-176b/1/.ipynb_checkpoints/
-rw-rw-r-- ec2-user/ec2-user 2112 2022-10-10 16:49 bloom-176b/1/.ipynb_checkpoints/model-checkpoint.py
-rw-rw-r-- ec2-user/ec2-user   24 2022-10-10 16:49 bloom-176b/1/.ipynb_checkpoints/model-checkpoint.pt
-rw-rw-r-- ec2-user/ec2-user   24 2022-10-12 22:58 bloom-176b/1/model.pt
drwxrwxr-x ec2-user/ec2-user    0 2022-10-10 17:11 bloom-176b/.ipy

In [264]:
!aws s3 cp ./model.tar.gz s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b  
    
    

upload: ./model.tar.gz to s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b


#### Start creating the end point

In [265]:
import sagemaker
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.utils import name_from_base
import boto3
region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")
import sagemaker
import boto3

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
region = boto3.Session().region_name

sm_client = boto3.Session().client(service_name="sagemaker", region_name=region)
s3_client = boto3.Session().client(service_name="s3", region_name=region)

In [266]:
s3_model_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path="./model.tar.gz",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b",
    sagemaker_session=session
)
s3_mme_model_path='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b'
print(s3_model_path_triton)
print(s3_mme_model_path)

s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b/model.tar.gz
s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b


In [267]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
from sagemaker.utils import name_from_base

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
    account_id=account_id_map[region], region=region, base=base
)
print(triton_image_uri)

max_seq_length=64

785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3


In [268]:
from sagemaker.utils import name_from_base

In [269]:
endpoint_name_p5 = name_from_base(f"bloom-176b-")
print(endpoint_name_p5)

container_p5 = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_triton,
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'bloom-176b',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response = sm_client.create_model(
    ModelName=endpoint_name_p5, ExecutionRoleArn=role, PrimaryContainer=container_p5
)
print(create_model_response)

bloom-176b--2022-10-13-00-11-16-799
{'ModelArn': 'arn:aws:sagemaker:us-east-1:622343165275:model/bloom-176b--2022-10-13-00-11-16-799', 'ResponseMetadata': {'RequestId': '10839bec-e854-405e-8a4c-48293d86f6e5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '10839bec-e854-405e-8a4c-48293d86f6e5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '97', 'date': 'Thu, 13 Oct 2022 00:11:16 GMT'}, 'RetryAttempts': 0}}


In [270]:

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_name_p5,
    ProductionVariants = [{
        'InstanceType'        : 'ml.g4dn.xlarge',
        'InitialVariantWeight': 1,
        'InitialInstanceCount': 1,
        'ModelName'           : endpoint_name_p5,
        'VariantName'         : 'AllTraffic'}])

create_endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:622343165275:endpoint-config/bloom-176b--2022-10-13-00-11-16-799',
 'ResponseMetadata': {'RequestId': 'a21b169d-0e7f-4e18-97ca-f7de1cdd221a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a21b169d-0e7f-4e18-97ca-f7de1cdd221a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Thu, 13 Oct 2022 00:11:16 GMT'},
  'RetryAttempts': 0}}

In [271]:
# Create the end point
create_endpoint_response = sm_client.create_endpoint(
    EndpointName         = endpoint_name_p5,
    EndpointConfigName   = endpoint_name_p5)

create_endpoint_response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:622343165275:endpoint/bloom-176b--2022-10-13-00-11-16-799',
 'ResponseMetadata': {'RequestId': '269dcb94-2345-4746-a35b-ddf609e11827',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '269dcb94-2345-4746-a35b-ddf609e11827',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 13 Oct 2022 00:11:16 GMT'},
  'RetryAttempts': 0}}

In [272]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint/bloom-176b--2022-10-13-00-11-16-799
Status: InService


In [273]:
%%time
import json

text = """This is a creative writing exercise. Below, you'll be given a prompt. Your story should be based on the prompt.

Prompt: A scary story about a haunted mouse
Story: On a dark and stormy night, the mouse crept in the shadows. """

text='this is a sample 1 line '

text = [1]*512
payload = {
    "inputs": [
        {
            "name": "INPUT__0",
            "data": text,
            "do_sample": True,
            "temperature": 0.7,
            "max_new_tokens":200,
            #"min_tokens": 100,
            "repetition_penalty": 1.1,
            "top_p": 500,
            'datatype':"INT32",
            'shape':[1, 512],

        }

    ]
}


response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="text/csv", 
    Body=json.dumps(payload),
)

print(json.loads(response["Body"].read().decode("utf8")))


{'model_name': 'bloom-176b', 'model_version': '1', 'outputs': [{'name': 'OUTPUT__0', 'datatype': 'INT32', 'shape': [512], 'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install nvidia-pyindex
!pip install tritonclient[http]
!pip install torch

In [132]:
import numpy as np
from transformers import T5Tokenizer

import tritonclient.http as httpclient

tokenizer = T5Tokenizer.from_pretrained("t5-small")
# print(tokenizer)
input_ids = tokenizer("summarize: SageMaker enables customers to deploy a model using custom code with NVIDIA Triton Inference Server. This functionality is available through the development of Triton Inference Server Containers. These containers include NVIDIA Triton Inference Server, support for common ML frameworks, and useful environment variables that let you optimize performance on SageMaker. For a list of all available Deep Learning Containers images, see Available Deep Learning Containers Images. Deep Learning Containers images are maintained and regularly updated with security patches.", return_tensors='pt').input_ids
print(input_ids.numpy().astype(np.int32).shape)

(1, 115)


In [ ]:
# set for http client

input_data = input_ids.numpy().astype(np.int32)

input_name = 'input'
output_name = "output"
inputs = []
outputs = []
inputs.append(httpclient.InferInput(input_name, input_data.shape, "INT32"))
inputs[0].set_data_from_numpy(input_data, binary_data=True)
outputs.append(
    httpclient.InferRequestedOutput(output_name, binary_data=True))
request_body, header_length = httpclient.InferenceServerClient.generate_request_body(inputs, outputs=outputs)

request_body_input, header_length = httpclient.InferenceServerClient.generate_request_body(inputs)

print(header_length)

request_body_input

In [ ]:
response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name_p5,
                                  ContentType='application/vnd.sagemaker-triton.binary+json;json-header-size={}'.format(header_length),
                                  Body=request_body,
                                 TargetModel='model.tar.gz')

# Parse json header size length from the response
header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
header_length_str = response['ContentType'][len(header_length_prefix):]

# Read response body
result = httpclient.InferenceServerClient.parse_response_body(
    response['Body'].read(), header_length=int(header_length_str))
output_data = result.as_numpy(output_name)

In [ ]:
output_data

In [ ]:
decoded_output = tokenizer.decode(
            output_data[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False,
        )
decoded_output

## Clean up

In [261]:
sm_client.delete_endpoint(EndpointName=endpoint_name_p5)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:622343165275:endpoint/bloom-176b--2022-10-13-00-01-27-641".